<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/chat_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import sqlite3
from tabulate import tabulate
import datetime

db = 'chat.db'

def show_tables():
  tables = ['users', 'chats', 'chat_users', 'messages']
  def show(tablename):
    try:
      with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM {tablename}')
        results = cursor.fetchall()
        headers = []
        for i in range(len(cursor.description)):
          headers.append(cursor.description[i][0])
        print(tablename)
        print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
        print("\n")
    except:
      print("Empty")

  for table in tables:
    show(table)

show_tables()

users
+------+--------+-------+
|   id |  name  |   age |
+======+========+=======+
|    1 |  Alex  |    23 |
+------+--------+-------+
|    2 |  Ann   |    22 |
+------+--------+-------+


chats
+------+--------------+
|   id |     name     |
+======+==============+
|    1 |   new_chat   |
+------+--------------+
|    2 | another_chat |
+------+--------------+


chat_users
+-----------+-----------+
|   chat_id |   user_id |
+===========+===========+
|         2 |         1 |
+-----------+-----------+
|         1 |         2 |
+-----------+-----------+
|         2 |         2 |
+-----------+-----------+


messages
+------+-----------+-----------+-----------+---------------------+
|   id |   chat_id |   user_id |  message  |     created_at      |
+======+===========+===========+===========+=====================+
|    1 |         1 |         1 |    hi     | 2023-11-16 18:14:34 |
+------+-----------+-----------+-----------+---------------------+
|    2 |         2 |         1 |    hi     

In [55]:
def create_db():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(
        """
          CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER
          );
        """
    )
    cursor.execute(
        """
          CREATE TABLE IF NOT EXISTS chats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT
          );
        """
    )
    cursor.execute(
        """
          CREATE TABLE IF NOT EXISTS chat_users (
            chat_id INTEGER,
            user_id INTEGER,
            PRIMARY KEY (chat_id, user_id),
            FOREIGN KEY (chat_id) REFERENCES chats (id) ON DELETE CASCADE,
            FOREIGN KEY (user_id) REFERENCES users (id)
          );
        """
    )
    cursor.execute(
        """
          CREATE TABLE IF NOT EXISTS messages (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            chat_id INTEGER,
            user_id INTEGER,
            message TEXT,
            created_at DATETIME
          );
        """
    )
    connection.commit()



def create_user(name, age):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("INSERT INTO users (name, age) VALUES (?, ?)", (name, age))
    connection.commit()
    cursor.close()
    connection.close()


def create_chat(name):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("INSERT INTO chats (name) VALUES (?)", (name,))
    connection.commit()
    cursor.close()
    connection.close()


def add_user_to_chat(username, chat_name):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("""
        SELECT c.id, u.id
        FROM chats AS c
        JOIN users AS u ON c.name = ? AND u.name = ?
    """, (chat_name, username))
    chat_id, user_id = cursor.fetchone()
    cursor.execute("INSERT INTO chat_users (chat_id, user_id) VALUES (?, ?)", (chat_id, user_id))

    connection.commit()
    cursor.close()
    connection.close()


def send_message(username, chat_name, message):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("""
        SELECT c.id, u.id
        FROM chats AS c
        JOIN users AS u ON c.name = ? AND u.name = ?
    """, (chat_name, username))
    chat_id, user_id = cursor.fetchone()
    cursor.execute("INSERT INTO messages (chat_id, user_id, message, created_at) VALUES (?, ?, ?, CURRENT_TIMESTAMP)", (chat_id, user_id, message))
    connection.commit()
    cursor.close()
    connection.close()


def delete_user(username):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("SELECT id FROM users WHERE name = ?", (username,))
    user_id = cursor.fetchone()[0]
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    connection.commit()
    cursor.close()
    connection.close()


def delete_chat(chat_name):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("SELECT id FROM chats WHERE name = ?", (chat_name,))
    chat_id = cursor.fetchone()[0]
    cursor.execute("DELETE FROM chats WHERE id = ?", (chat_id,))
    connection.commit()
    cursor.close()
    connection.close()


def delete_user_from_chat(username, chat_name):
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("""
        SELECT c.id, u.id
        FROM chats AS c
        JOIN users AS u ON c.name = ? AND u.name = ?
    """, (chat_name, username))
    chat_id, user_id = cursor.fetchone()
    cursor.execute("DELETE FROM chat_users WHERE chat_id = ? AND user_id = ?", (chat_id, user_id))
    connection.commit()
    cursor.close()
    connection.close()


def get_users():
    connection = sqlite3.connect("chat.db")
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM users")



# create_db()

# create_user("Alex", 23)
# create_user("Ann", 22)
# create_chat("new_chat")
# create_chat("another_chat")
# add_user_to_chat("Alex", "new_chat")
# add_user_to_chat("Alex", "another_chat")
# add_user_to_chat("Ann", "new_chat")
# add_user_to_chat("Ann", "another_chat")
# send_message("Alex", "new_chat", "hi")
# send_message("Alex", "another_chat", "hi")
# send_message("Ann", "new_chat", "hi")
# send_message("Ann", "another_chat", "hi")


# delete_user("Alex")
delete_user_from_chat("Alex", "new_chat")